In [1]:
from pyspark.sql import SparkSession

# สร้าง SparkSession
spark = SparkSession.builder \
    .appName("MongoDBIntegration") \
    .master("local")\
    .config("spark.jars", 
            r"C:\Users\chopp\mongo-spark-connector_2.12-10.4.0-all.jar,"
            r"C:\Users\chopp\mongodb-driver-core-5.2.0-javadoc.jar,"
            r"C:\Users\chopp\bson-5.2.0-javadoc.jar") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/mydb") \
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/mydb") \
    .getOrCreate()

# อ่านข้อมูลจาก MongoDB
df = spark.read \
.format("mongodb") \
.option("uri", "mongodb://localhost:27017/mydb") \
.option("database", "mydb") \
.option("collection", "employees") \
.load()
df.show()


+--------------------+-------------+------------+-------------------+----------+------+--------------------+
|                 _id|      address|  department|            general|      name|salary|              social|
+--------------------+-------------+------------+-------------------+----------+------+--------------------+
|61335a62f6c9d148a...|กรุงเทพมหานคร| ฝ่ายการตลาด| {60.0, 170.0, ชาย}|      ก้อง| 60000|[facebook, line, ...|
|6133601501d011636...|       ชลบุรี|โปรแกรมเมอร์| {70.5, 178.6, ชาย}|     สมชาย| 40000|    [facebook, line]|
|613362ea01d011636...|กรุงเทพมหานคร|     ฝ่ายขาย|{49.0, 165.0, หญิง}|    สกาวใจ| 15000|    [facebook, line]|
|6133640201d011636...|กรุงเทพมหานคร|      วิศวกร| {49.0, 168.0, ชาย}|   ญัฐวุติ| 45000|[facebook, line, ...|
|6133643801d011636...|        ระยอง|     ฝ่ายขาย|{49.0, 168.0, หญิง}| สุดารัตน์| 15000|[facebook, line, ...|
|613365b101d011636...|        ระยอง|โปรแกรมเมอร์| {52.0, 175.0, ชาย}|    คมกริช| 40000|[facebook, line, ...|
|6133673801d011636.